# Mining Frequent Itemsets

In this notebook, we consider the problem of mining frequent itemsets when the input is a collection of transactions, where a transaction is again an itemset. The frequent itemsets we want to mine from the transactions will be subsets of these transactions, and so we are interested in discovering the most frequent subsets in such transactions. We are going to present two widely used algorithms: the *A-Priori algorithm* and the *FP-Growth algorithm*. The last one is implemented in the machine learning library of spark.

Preliminary start-up code:

In [1]:
import pyspark
import os
import math
import random
import sys

# make sure pyspark tells workers to use python2 not 3 if both are installed\n",
os.environ["PYSPARK_PYTHON"] = "python3"
#os.environ['PYTHONPATH'] = ':'.join(sys.path)

In [2]:
spark_home = os.environ.get('SPARK_HOME', None)

In [3]:
print ( spark_home )

None


In [4]:
sc = pyspark.SparkContext('local[*]')

In [5]:
print ( sc )

<SparkContext master=local[*] appName=pyspark-shell>


The context where this problem has been defined and studied extensively is the analysis of costumer habits in traditional retail stores. We consider mining frequent itemsets from a sequence of sets, these sets are traditionally called transactions or baskets, because the original application domain where this problem has been studied has been discovering frequently bought together products (items) in different purchases (baskets or transactions). In this context, given a frequency threshold $\theta$, and a sequence of sets $S_1,S_2,...,S_M$, we say that a itemset $P \subseteq I$ is frequent if its support among  $S_1,S_2,...,S_M$ is $ \geq \theta  $, where the support is the fraction of sets that contain $P$. As we will see when we talk of recommender systems, in the case of recommending products for on-line shops where many clients buy many different things, the kinds of problems considered are different to this frequent itemset problem.

For a tradicional retail store, finding frequent itemsets can be important towards directing their marketing efforts towards sets of products that **many** costumers will find interesting, but in the setting of on-line shops, the recommendations can be (or **should be**) personalized for each costumer, so this makes the domains (tradicional and on-line) different regarding the kind of problems they want to solve for finding marketing strategies.  However, even for on-line stores we can find sometimes some small subsets of products that are bought together very frequently (e.g. people that buy iPhones may tend to buy particular protective cases).


First, we are going to present the most well-known algorithm for solving this problem, the A-Priori algorithm. It is not the most efficient nowadays, for many cases, but it is a simple algorithm that shows clearly the difficulties in solving this problem. We will also discuss the idea behind a more advanced algorithm, the FP-Growth algorithm, that has also an specific efficient parallel version in the map-reduce framework that it is implemented in the machine library of spark. 

## The A-Priori Algorithm

This algorithm is based on exploiting the following basic principle, the monotonicity of itemsets:

         If a set P of items is frequent, then so is every subset of P
            
Observe that if this is true for $P$ with frequency $\theta$, it will be true for every subset of $P$ with frequency *at least* $\theta$. The typical situation will be that for a fixed frequency $ \theta $, we will find much less frequent sets of size $k$ than of size $k-1$. In the particular domain of tradicional (brick-and-mortar) retail stores, the typical transaction size will be small, so when considering high frequency thresholds, we will not find many frequent item sets of big size.  

So, the A-Priori algorithm is based on first finding smaller frequent-items sets, and then going to the next size but considering only sets such that any of their subsets has been found previously to be frequent. This suggests an iterative algorithm that first looks for frequent itemsets of size 1, then of size 2, and so on.

Here you have a very high-level pseudo-code of the A-Priori algorithm:

>Find frequent singleton-sets (k=1), store them in $L_1$  
>k=2  
>While ($L_{k-1}$ is not empty) do:  
    >>Define $C_k$ to be the set of itemsets of size k, where every subset of size $k-1$ is in $L_{k-1}$  
    >>For every transaction B, find all its itemsets of size k that are in $C_k$  
    >>Add to $L_k$ any the previous itemsets of size $k$ that has been found in at least a $\theta$ fraction of the transactions    
    >>k=k+1
>

Where is the bottleneck performance in a possible implementation of this algorithm ?

A practical (usable) implementation of the general A-Priori algorithm should use specialized data structures to low the complexity of the candidate set generation and filtering steps. However, we here consider an implementation for the particular case of only computing $L_1$ and $L_2$ sets. I include here the full code for computing L1, but for L2 I only include a partial code, that **YOU should finish it as a programming exercise**.

In [6]:
# From a transaction in a single string with items separated by white spaces
# generate a list with the items of the transaction
def parseTransaction( trans ):
    return trans.strip().split(' ')

# Compute the rdd with frequent singletonsets (L_1)
def computeL1 ( rddtrans, numtrans, theta ):
  rddtemp = rddtrans.flatMap( lambda trans : [ (it,1) for it in trans ] ).reduceByKey( lambda a,b : a+b  )
  rddL1 = rddtemp.filter( lambda x : (float(x[1])/numtrans) >= theta )
  return rddL1

# This is a function to map transactions to transactions with only items in L1,
# something that can increase the efficiency when computing L2
def filterOutL1( transseq, L1 ):
    for trans in transseq:
       yield [ it for it in trans if (it in L1) ]



### Exercise: Finish the implementation of the A-priori algorithm for k up to 2

In [ ]:
#
#      HERE ARE THE TWO FUNCTIONS YOU SHOULD FINISH
# 

# Compute L2 from transactions and L1. This function assumes
# that the input rddtrans has been simplified by eliminating from the transactions those items not in L1
# Also, a generalized version of this function for any size k should use
# more specialized data structures to test-and-check candidate itemsets for L_k from L_{k-1}

# First, the function to generate candidate frequent pairs (C_2)
# for each transaction of a partition of the set of transactions
# This function expects a sequence of transactions, so it should be used with 
# mapPartitions
#
def generateC2( seqoftransactions,  L1  ):
    for trans in seqoftransactions:
      itemsetlist = []
      #
      #   INSERT CODE HERE for computing all the pairs (a,b) of items
      #    in trans such that both iterms belong to L1 and store them
      #   in itemsetlist as (key,value) pairs of the form  ((a,b),1)
      yield itemsetlist
    
# And finally, the function for computing L_2 from L_1 and the rdd with
# the set of transactions
# This function should use your previous generateC2 function to compute L2 from C2
#
def computeL2( rddtrans, numtrans, L1, theta ):
    #   Map the transactions in rddtrans to the set of candidate frquenquent pairs (C2) 
    #   using the previous function generateC2
    candidatePairsrdd = 
    #  Count number of occurences in the transactions for  each pair in C2. 
    #  Use flatpMap and reduceByKey to get the final rdd
    rddL2temp = 
    # Finally, filter out from the previous rdd those pairs with frequency below theta
    rddL2 = 
    return rddL2


Let's try it with the sample set in the file "sample_fpgrowth.txt" :

> r z h k p  
> z y x w v u t s   
> s x o n r  
> x z y m t s q e  
> z  
> x z y r q t p  


In [ ]:
#
# Once you finish your code, test it with this cell and the next one.
# Load set of transactions as a sequence of itemsets (every transaction stored in a list)
#
rddtrans = sc.textFile( spark_home+"/data/mllib/sample_fpgrowth.txt" ).map(parseTransaction)
numtrans = rddtrans.count()


# First, let's compute the frequent singleton sets with theta=0.25
#
rddL1 = computeL1( rddtrans, float(numtrans), 0.25  )
L1 = rddL1.collect()
print ("L1 together with frequency information: \n", L1 )

# Compute a version without the frequency information:
#
L1onlyitems = rddL1.map( lambda it : it[0] ).collect()
print ( "L1 only items: \n", L1onlyitems )

Once you finish the code for functions generateC2 and computeL2, you can try execute the next code cell to compute the set of frequent pairs (L2). For the sample file , this is the set L2 you should get (for frequency 0.25):

> pairs with frequency information :  [((u't', u'x'), 3), ((u'p', u'r'), 2), ((u't', u'z'), 3), ((u's', u'y'), 2), ((u'r', u'z'), 2), ((u'x', u'z'), 3), ((u'q', u'y'), 2), ((u'r', u'x'), 2), ((u'p', u'z'), 2), ((u's', u'x'), 3), ((u'y', u'z'), 3), ((u's', u'z'), 2), ((u'q', u'z'), 2), ((u'q', u'x'), 2), ((u'q', u't'), 2), ((u's', u't'), 2), ((u'x', u'y'), 3), ((u't', u'y'), 3)]

In [ ]:
# Next, compute frequent pairs (L2) from frequent items (L1)
# We  need to work with the collected back version (to the driver) of the L1 set
# (This will be Ok as far as L1 is small enough to fit in the memory
# of every single machine ). We use mapPartitions to distribute only one function call per partition,
# avoiding the overhead of executing many functions with the parameter L1
#
L1filteredtransrdd = rddtrans.mapPartitions( lambda transseq : filterOutL1( transseq, L1onlyitems )  )

# Let's check if the filtered transactions are correct:
#
print (" transactions with only L1 items : ", L1filteredtransrdd.collect() )

# Next, compute the frequent pairs (L_2)
#
rddL2 = computeL2( L1filteredtransrdd, float(numtrans), L1onlyitems, 0.25 )
print ( "\n pairs with frequency information : ", rddL2.collect() )

## Finding Association Rules

The original application domain of the frequent itemsets problem was finding good marketing strategies in traditional retail stores. For example, in a famous application of this problem, it was found that a frequent itemset in costumer transactions was the pair {diapers,beer}, but that the frequency of the singleton set {beer} in the whole set of transactions was much smaller than the frequency of  {diaper,beer} in the transactions with {diapers}. We can quantify this situation with two measures associated with the rule. We say that the rule {diapers} -> {beer} has a high **confidence** if the ratio:

> support( {diapers,beer} ) / support( {diapers})                        
                           
is high (the fraction of itemsets with diapers that also contain beer is high). But observe that this alone does not mean that the rule {diapers} -> {beer} shows a *true relationship*. That is, it could happen that all the costumers buy beer, so the  rule {diapers} -> {beer} does not really provide an useful information to discover when people buys **more beer**. To quantify this situation, we need also to quantify the **interest** of the rule {I} -> {j} as the difference between its confidence and the frequency of {j}:

> confidence ( {diapers} -> {beer} ) - frequency( {beer} ) 

Observe that the interest can be positive, zero o negative. For the diapers-beer example mentioned, the interest was positive, meaning that when diapers was present the relative frequency of beer was higher than when looking for beer in all the transactions. A negative interest would indicate a negative effect, i.e. that when {I} is present {j} is less likely to be present than in general.

So, towards building such marketing recommendation system, it can be interesting to advance one more step and build association rules that are interesting for this final marketing task. For example, given a set of frequent itemsets, we can compute the confidence and interest of any association rule of the kind {I} -> {j} we can build with the frequent itemsets. From a given frequent itemset P, observe that we can build |P|-1 different association rules. 

To consider the difference between the confidence and the interest of an association rule consider the two following transaction sets:

In [7]:
beeranddiapers1 = [['beer','diapers','cheese'],
                   ['beer','diapers','pizza'],
                   ['beer','diapers','pizza'],
                   ['beer','diapers','milk'],
                   ['beer','diapers','milk','pizza'],
                   ['beer','diapers','toothpaste'],
                   ['beer','diapers','icecream'],
                   ['beer','diapers','pizza','yogurt']]

beeranddiapers2 = [['beer','diapers','cheese'],
                   ['beer','diapers','pizza'],
                   ['beer','diapers','pizza'],
                   ['beer','diapers','milk'],
                   ['yogurt','milk','pizza'],
                   ['pizza','toothpaste'],
                   ['yogurt','icecream'],
                   ['beer','pizza','yogurt']]


We have that the confidence for the rule: 
> {diapers} -> {beer}

  is:

> $ \frac{support(\{beer,diapers\})}{support(\{diapers\})} = \frac{8}{8} = 1 $  (for the first set)  
> $ \frac{support(\{beer,diapers\})}{support(\{diapers\})} = \frac{4}{4} = 1 $  (for the second set)  

So, for both sets the confidence of the rule is maximum. However, we observe that the rule is really more useful for the second transaction set because the interest for the first transaction set is:

> $ confidence ( \{diapers\} \rightarrow \{beer\} ) - frequency( \{beer\} ) = 1 - 1 = 0$

but for the second set is: 

> $ confidence ( \{diapers\} \rightarrow \{beer\} ) - frequency( \{beer\} ) = 1 - (5/8) = 0.375 $

That is, in the second transaction set the presence of diapers makes more likely the presence of beer


### Exercise

Consider the following programming exercise. Given the information of the frequent singletons and frequent pairs sets we generated with our previous implementation, implement the following functions to generate the association rules linked with the frequent itemsets together with its confidence and its interest.

In [ ]:
#
#  Given a:
#    - a frequent pair with its support information  like (('t', 'x'), 3)
#    - an item j from the pair like  't'
#    - the number of transactions
#    - a list with the L1 information computed with the function computeL1 and then collected back
#    to the driver
#
#   Compute the confidence and interest of the rule  freqitemset-{j} ->  {j}
#   Return it with the format:  ( 'freqitemset-{j} ->  j', confidence, interest )
def compConfidenceAndInterestForRule( freqpairWithFreq, j, numtrans, L1withfreqinfo ):
    #
    # INSERT YOUR SOLUTION HERE
    #
    premise = 
    confidence =     
    freqj = 
    interest = confidence - freqj
    return ( str(premise)+" -> "+str(j), confidence, interest  )
    

#
#    Map each freqpair of a partition, to its SET of different association rules
#    together with their confidence and interest
#
#    This function should use the previous compConfidenceAndInterestForRule for each
#    association rule generated
#
def genAssocRulesForPartition( freqitemsetsequence, numtrans, L1withfreqinfo ):
    for fpair in freqitemsetsequence:
        rulesforpair = []
        # INSERT YOUR SOLUTION HERE    
        yield rulesforpair 

#
#   Finally, map all the frequenitemsets of all the partitions of the input
#   rddfreqsets using the previous function, and finally collect back all
#   the association rules obtained, but show them on the screen ordered by
#   descending order, using their interest to orden them
#
def mapFreqItemSetsToAssocRules( rddfreqsets, numtrans, L1withfreqinfo ):
    # Compute an rdd with the assotiation rules for the freqsets, calling the previous function
    # for each partition of the rddfreqsets 
    #   INSERT CODE HERE 
    #
    # Next, get a "flat" version of the previous set of rules (one rule per element of the RDD)
    #   INSERT CODE HERE
    #
    # Next, collect back the obtained association rules, but sort them in descending order by their interest
    # You can either sort in a new rdd and then collect back the sorted rdd, or sort them in the driver after
    # collecting back the set of association rules.
    #
    #   INSERT YOUR SOLUTION HERE
    
    

In [ ]:
#
# Next, test your code with the RDD of frequent pairs rddL2 computed before
# and its corresponding set of frequent singletons L1
#
#   INSERT YOUR SOLUTION HERE:


# You can try your code also with the transaction sets beeranddiapers1 and beeranddiapers2
# (Compute first L1 and L2 for them for computing their association rules)


## The FP-Growth Algorithm

We have discussed in the A-Priori frequent itemsets mining algorithm that a major bottleneck in the performance is the possible increasing number of candidate itemsets as the size $k$ considered increases in every stage of the algorithm. We are next going to present the basic idea behind an algorithm, the FP-Growth, that avoids this candidate set generation of itemsets of size k ($C_k$) to be able to generate the actual set of frequent itemsets of size k ($L_k$). The algorithm achieves this improvement thanks to a data structure called a **prefix-search tree**. This tree is a *compact* representation of the set of transactions, but in such a way that allows a traversal of its branches to efficiently find the frequent itemsets of any size k scanning the tree only once.

Consider the following set of transactions:

> f c a m p i   
> f c a b m o   
> f b  j  
> c b p s   
> f c a m p l    


Then, its corresponding prefix-search tree, when considering minSuport=3/5 and its corresponding ordered set of singleton sets L1 with support at least 3/5: $ \{ f:4, c:4, a:3, b:3, m:3, p:3 \} $,  is the following one:

<img src="exampleFPtree.png" width="250px" />

This prefix-search tree represents in a compact way the four transactions that contain the most frequent element (f), with the subtree with root f. Observe that this subtree contains three subbranches (that share some nodes), where the branch f-c-a-m-p encodes the information of the first and last transactions of the DB, the branch f-c-a-b-m encodes the second transaction, and the branch f-b the third transaction. Finally, the branch c-b-p represents the fourth transaction. The number that labels each node represents the number of transactions of the DB that contain the subset given by the path of items from the root to the node.

Given a DB of transactions and the ordered set L1 computed with a particular minSupport, the following algorithm (pseudo-code) can be used to build such prefix-search tree

```python
 def buildFPtree( L1, DB ):
   # create initial tree with only a root
   fptree = (root,())
   L1 = sortByFrequency(L1)
   for tran in DB:
      sortedtran = sortByFrequency(tran,L1)
      insertTree(sortedtran,fptree)
   return fptree
   
 def insertTree( sortedtran, fptree ):
   if (sortedtran is empty):
      return
   firstit = head(sortedtran)
   if (fptree has child node with name firstit):
       # get subtree with root named firstit
       fpsubtree = getsubtree( fptree, firstit )
       incrementRootCounter( fpsubtree )
   else:
      # create new subtree an attach it as child of fptree
      fpsubtree = ((firstit,1),())
      addchild(fptree,fpsubtree)
   # process remaining items of the transaction over the subtree
   tailtran = sortedtran.delete(firstit)
   insertTree( tailtran, fpsubtree )   
   
```


Observe that the overall process for building the prefix-search tree needs two scans of the transaction DB, one for building the ordered set L1, and a second one for building the tree from L1 and the DB of transactions, where each transaction is processed once.


Once we have such tree, it is possible to compute the **full set of frequent subsets** with the given minSupport, from our DB of transactions with a recursive algorithm that traverses the tree recursively. The details of such algorithm (and of the previous prefix-search tree construction) can be found in the following paper:


>  Jiawei Han, Jian Pei, Yiwen Yin, Runying Mao. *Mining Frequent Patterns without Candidate Generation: A Frequent-Pattern Tree Approach*. Data Mining Knowledge Discovery. 8(1): 53-87 (2004).
> Link: http://hanj.cs.illinois.edu/pdf/dami04_fptree.pdf


Spark includes a distributed version of FPgrowth, that works by creating different independent prefix-search trees in different partitions and finally merging the resulting frequent subsets of each partition, described in the paper:

>  Haoyuan Li, Yi Wang, Dong Zhang, Ming Zhang, Edward Y. Chang. *PFP: parallel FP-growth for query recommendation*. RecSys 2008: 107-114.
> Link: http://dl.acm.org/citation.cfm?doid=1454008.1454027


Let's see this algorithm in action when looking for all the frequent subsets of the sample file sample_fpgrowth.txt

In [7]:
from pyspark.mllib.fpm import FPGrowth

data = sc.textFile( spark_home+"/data/mllib/sample_fpgrowth.txt")
transactions = data.map(lambda line: line.strip().split(' '))
model = FPGrowth.train(transactions, minSupport=0.25, numPartitions=4)

# Collect back the set of ALL frequent itemsets with minSupport
# Beware !, this set could be exponientally large with respect to the set of
# input transactions !
#
rddfreqitemsets = model.freqItemsets()
result = rddfreqitemsets.collect()
for fi in result:
    print( fi )

FreqItemset(items=['r'], freq=3)
FreqItemset(items=['r', 'x'], freq=2)
FreqItemset(items=['r', 'z'], freq=2)
FreqItemset(items=['z'], freq=5)
FreqItemset(items=['s'], freq=3)
FreqItemset(items=['s', 't'], freq=2)
FreqItemset(items=['s', 't', 'x'], freq=2)
FreqItemset(items=['s', 't', 'x', 'z'], freq=2)
FreqItemset(items=['s', 't', 'z'], freq=2)
FreqItemset(items=['s', 'x'], freq=3)
FreqItemset(items=['s', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y'], freq=2)
FreqItemset(items=['s', 'y', 't'], freq=2)
FreqItemset(items=['s', 'y', 't', 'x'], freq=2)
FreqItemset(items=['s', 'y', 't', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y', 't', 'z'], freq=2)
FreqItemset(items=['s', 'y', 'x'], freq=2)
FreqItemset(items=['s', 'y', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y', 'z'], freq=2)
FreqItemset(items=['s', 'z'], freq=2)
FreqItemset(items=['x'], freq=4)
FreqItemset(items=['x', 'z'], freq=3)
FreqItemset(items=['t'], freq=3)
FreqItemset(items=['t', 'x'], freq=3)
FreqItemset(items=['t', 'x',

However, because model.freqItemsets() is a RDD, we can instead save them to a data file, or filter only some of them to be analyzed by our program back in the driver application:

In [8]:
pairs = rddfreqitemsets.filter( lambda x  : len(x.items) == 2  ).collect()

print ( "Frequent pairs: " )
for fi in pairs:
    print( fi )
    
print ( "\nFrequent triples: " )
triples = rddfreqitemsets.filter( lambda x  : len(x.items) == 3  ).collect()    
for fi in triples:
    print ( fi )


Frequent pairs: 
FreqItemset(items=['r', 'x'], freq=2)
FreqItemset(items=['r', 'z'], freq=2)
FreqItemset(items=['s', 't'], freq=2)
FreqItemset(items=['s', 'x'], freq=3)
FreqItemset(items=['s', 'y'], freq=2)
FreqItemset(items=['s', 'z'], freq=2)
FreqItemset(items=['x', 'z'], freq=3)
FreqItemset(items=['t', 'x'], freq=3)
FreqItemset(items=['t', 'z'], freq=3)
FreqItemset(items=['p', 'r'], freq=2)
FreqItemset(items=['p', 'z'], freq=2)
FreqItemset(items=['q', 't'], freq=2)
FreqItemset(items=['q', 'x'], freq=2)
FreqItemset(items=['q', 'y'], freq=2)
FreqItemset(items=['q', 'z'], freq=2)
FreqItemset(items=['y', 't'], freq=3)
FreqItemset(items=['y', 'x'], freq=3)
FreqItemset(items=['y', 'z'], freq=3)

Frequent triples: 
FreqItemset(items=['s', 't', 'x'], freq=2)
FreqItemset(items=['s', 't', 'z'], freq=2)
FreqItemset(items=['s', 'x', 'z'], freq=2)
FreqItemset(items=['s', 'y', 't'], freq=2)
FreqItemset(items=['s', 'y', 'x'], freq=2)
FreqItemset(items=['s', 'y', 'z'], freq=2)
FreqItemset(items=['t

In addition, you can also use the function RDD.toLocalIterator() that given a RDD allows you to iterate though all the elements of the RDD in the driver program. So in case you want to perform some individual final task on each resulting frequent subset you could use it

In [9]:
for fset in rddfreqitemsets.toLocalIterator():
    print ( " Frequent subset: ", fset )

 Frequent subset:  FreqItemset(items=['r'], freq=3)
 Frequent subset:  FreqItemset(items=['r', 'x'], freq=2)
 Frequent subset:  FreqItemset(items=['r', 'z'], freq=2)
 Frequent subset:  FreqItemset(items=['z'], freq=5)
 Frequent subset:  FreqItemset(items=['s'], freq=3)
 Frequent subset:  FreqItemset(items=['s', 't'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 't', 'x'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 't', 'x', 'z'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 't', 'z'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 'x'], freq=3)
 Frequent subset:  FreqItemset(items=['s', 'x', 'z'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 'y'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 'y', 't'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 'y', 't', 'x'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 'y', 't', 'x', 'z'], freq=2)
 Frequent subset:  FreqItemset(items=['s', 'y', 't', 'z'], freq=2)
 Frequent subset:  FreqItemset(items=[

The function toLocalIterator() maintains one partition into the driver each time, so the driver machine should have at least the same memory as any of the worker machines in your cluster.